In [2]:
import nltk, re
from nltk.stem.porter import PorterStemmer
import pandas as pd



# Reference: https://ijbnpa.biomedcentral.com/articles/10.1186/s12966-017-0525-8

# Try: need to have at least 1 key verb and 1 key noun to be classified as sedentary
# instead of relying on just 1 keyword

In [3]:
# test set
test = pd.read_json('~/Documents/PhD/HBDC/FilteredTweets/_tweets_2017.12.01.20.09.json', lines=True)

test

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,quoted_status_id,quoted_status_id_str,reply_count,retweet_count,retweeted,source,text,timestamp_ms,truncated,user
0,NaN,None,2017-10-29 22:43:39,"[0, 12]","{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,9.245877e+17,9.245877e+17,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",u inspire me https://t.co/VdECceNqpI,2017-10-29 22:43:39.375,False,"{u'follow_request_sent': None, u'profile_use_b..."
1,NaN,None,2017-10-29 22:43:39,"[0, 26]","{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,9.247346e+17,9.247346e+17,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",An icon of Canadian art... https://t.co/Fxk3w5...,2017-10-29 22:43:39.431,False,"{u'follow_request_sent': None, u'profile_use_b..."
2,NaN,None,2017-10-29 22:43:40,"[9, 32]","{u'user_mentions': [{u'id': 203123011, u'indic...",NaN,NaN,0,False,low,...,9.244376e+17,9.244376e+17,0,0,False,"<a href=""http://twitter.com/download/android"" ...",@CTVNews https://t.co/vcQUwe5ww6,2017-10-29 22:43:40.198,False,"{u'follow_request_sent': None, u'profile_use_b..."
3,NaN,None,2017-10-29 22:43:40,"[40, 49]","{u'user_mentions': [{u'id': 747981270, u'indic...",NaN,NaN,0,False,low,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",@rouxx21 @TalonFruitBat @Twilight_Saint &lt;33333,2017-10-29 22:43:40.311,False,"{u'follow_request_sent': None, u'profile_use_b..."
4,NaN,None,2017-10-29 22:43:39,"[0, 62]","{u'user_mentions': [{u'id': 1565023633, u'indi...",{u'media': [{u'expanded_url': u'https://twitte...,NaN,0,False,low,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",I get A's &amp; B's and she gets D's....nuts!!...,2017-10-29 22:43:39.434,False,"{u'follow_request_sent': None, u'profile_use_b..."
5,NaN,None,2017-10-29 22:43:40,"[27, 140]","{u'user_mentions': [{u'id': 4560764232, u'indi...",NaN,"{u'display_text_range': [27, 167], u'entities'...",0,False,low,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com/download/android"" ...",@_Nick_Nation_ @nickcarter Can believe its bee...,2017-10-29 22:43:40.770,True,"{u'follow_request_sent': None, u'profile_use_b..."
6,NaN,None,2017-10-29 22:43:40,"[0, 100]","{u'user_mentions': [{u'id': 567010052, u'indic...",{u'media': [{u'expanded_url': u'https://twitte...,NaN,0,False,low,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",Fav song 🔥 Honestly I’ve been playing this so...,2017-10-29 22:43:40.808,False,"{u'follow_request_sent': None, u'profile_use_b..."
7,NaN,None,2017-10-29 22:43:41,"[0, 0]","{u'user_mentions': [], u'symbols': [], u'hasht...",{u'media': [{u'expanded_url': u'https://twitte...,NaN,0,False,low,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com/download/android"" ...",https://t.co/dq4Ti7e1wx,2017-10-29 22:43:41.667,False,"{u'follow_request_sent': None, u'profile_use_b..."
8,NaN,"{u'type': u'Point', u'coordinates': [-66.0196,...",2017-10-29 22:43:42,NaN,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,NaN,NaN,0,0,False,"<a href=""https://untappd.com"" rel=""nofollow"">U...",It has more sweet flavour notes than most othe...,2017-10-29 22:43:42.172,False,"{u'follow_request_sent': None, u'profile_use_b..."
9,NaN,None,2017-10-29 22:43:42,"[0, 18]","{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,9.247512e+17,9.247512e+17,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",Correct.That’s it. https://t.co/0jtu2oILGN,2017-10-29 22:43:42.359,False,"{u'follow_request_sent': None, u'profile_use_b..."


In [4]:
# Verbs indicate sedentary
sedentary_verbs = ['watch', 'chill', 'work', 'study', 'lying', 'recline', 'sit\s', 'read\s', 'draw', 'paint'
        , 'drive']

# Nouns indicate sedentary
sedentary_nouns = ['game', 'video game', 'school', 'office', 'television', 'tv', 
         'computer', 'phone', 'iphone', 'tablet', 'ipad', 'kindle', 'book', 
         'newspaper', 'homework', 'car\s', 'plane', 'bus', 'train', 'netflix',
         'bed', 'the play', 'the show'] 



# list of sport leagues
sedentary_leagues = ['nba', 'worldseri', 'nfl', 'mls', 'baseball', 'mlb', 'soccer'
                     , 'mls', 'ufc', 'nhl', 'cfl']

sedentary_exclusion = ['look for work', 'hire', 'job', 'career']


# French version
from googletrans import Translator
translator = Translator()

sedentary_verbs_fr = [translator.translate(x, dest='fr').text for x in sedentary_verbs]
#sedentary_verbs_fr

sedentary_nouns_fr = [translator.translate(x, dest='fr').text for x in sedentary_nouns]
#sedentary_nouns_fr  


# stem forms
porter = nltk.PorterStemmer()

sedentary_verbs_stemmed = [porter.stem(item) for item in sedentary_verbs]
#sedentary_verbs_stemmed

# stemming nouns doesnt make sense
#sedentary_nouns_stemmed = [porter.stem(item) for item in sedentary_nouns]
#sedentary_verbs_stemmed




# Regex pattern
sedentary_verbs_regex = "\s(" + '|'.join(sedentary_verbs + sedentary_verbs_stemmed + sedentary_verbs_fr) + ')'



sedentary_nouns_regex = "\s(" + '|'.join(sedentary_nouns + sedentary_nouns_fr + sedentary_leagues) + ')'
#sedentary_nouns_regex = '\s' + '|'.join(sedentary_nouns + sedentary_nouns_stemmed)
#sedentary_nouns_regex = re.sub(' ', '\ ', sedentary_nouns_regex)


sedentary_exclusion_regex = "\s(" + '|'.join(sedentary_exclusion) + ')'




In [5]:
##### Physical Activity
# Verbs indicate Phyiscal Activity
activity_verbs = ['play', 'train', 'run\s', 'hike', 'ski\s', 'bike', 'lift']

# Nouns indicate Physical Activity
activity_nouns = [ 'gym\s', 'workout', 'work\ out', 'basketball', 'basket\ ball'
                  ,'hockey', 'biking', 'bb', 'bball', 'baseball', 'climb', 'dance', 'dancin'
                  ,'skate', 'skating', 'squat', 'weights', 'gains', 'volleyball', 'yoga'
                 , 'park', 'mountain'] 


# combine nouns with wikipedia sports
#FIX THE SPACE IN BETWEEN WORDS WITH \(space)
with open('sportslist.txt') as f:
    lines = f.read().splitlines()

    activity_nouns = activity_nouns + [x.lower() for x in lines if x.lower() not in activity_nouns] # removed 3 duplicates

# Exclusion
activity_exclusion = ['look for work', 'hire', 'job', 'career', 'game']


# # French
activity_verbs_fr = [translator.translate(x, dest='fr').text for x in activity_verbs]

with open('sportslist_fr.txt') as f:# too many to make google calls
    lines = f.read().splitlines()
activity_nouns_fr = [x.lower() for x in lines if x.lower() not in activity_nouns]


# stemmed form
activity_verbs_stemmed = [porter.stem(item) for item in activity_verbs]

# Regex pattern
activity_verbs_regex = "\s(" + '|'.join(activity_verbs + activity_verbs_stemmed + activity_verbs_fr) + ')'


activity_nouns_regex = "\s(" + '|'.join(activity_nouns + activity_nouns_fr) + ')'


activity_exclusion_regex = "\s(" + '|'.join(activity_exclusion) + ')'


temp = re.sub('a.k.a','',activity_nouns_regex)
temp = re.sub('\.','', temp)
temp = re.sub('/','',temp)
temp = re.sub('\[','',temp)
temp = re.sub('\]','',temp)
temp = re.sub('|game','',temp)
#print temp

activity_nouns_regex = temp


In [7]:
# tokenize each tweets
# look for match in keyword
# have to include word boundary before, and some need after as well

for i, item in enumerate(test.text):
    #print item 
    lowered = item.lower()
    
    tokens = nltk.word_tokenize(lowered)

    stemmed_tokens = [porter.stem(x) for x in tokens]

    #print stemmed_tokens
    
    ##### Sedentary 
    sedentary_verbs_count = len(re.findall(sedentary_verbs_regex, ' '.join(stemmed_tokens)))
    #print re.findall(verbs_regex, ' '.join(stemmed_tokens))
    #print verbs_count

    sedentary_noun_count = len(re.findall(sedentary_nouns_regex, ' '.join(stemmed_tokens)))
    #print re.findall(nouns_regex, ' '.join(stemmed_tokens))
    #print noun_count
    #print '\n'
    
    sedentary_exlusion_count = len(re.findall(sedentary_exclusion_regex, ' '.join(stemmed_tokens)))

    if sedentary_verbs_count >=1 and sedentary_noun_count >=1 and sedentary_exlusion_count == 0:
    	test.at[i, 'sedentary'] = 1
#     	print item
#     	#print stemmed_tokens
#     	print re.findall(sedentary_verbs_regex, ' '.join(stemmed_tokens))
#     	print re.findall(sedentary_nouns_regex, ' '.join(stemmed_tokens))
#     	print '\n'
    else:
    	test.at[i, 'sedentary'] = 0

    
    ##### Phyiscal Activity 
    activity_verbs_count = len(re.findall(activity_verbs_regex, ' '.join(stemmed_tokens)))

    activity_noun_count = len(re.findall(activity_nouns_regex, ' '.join(stemmed_tokens)))
 
    activity_exlusion_count = len(re.findall(activity_exclusion_regex, ' '.join(stemmed_tokens)))

    if activity_verbs_count >=1 and activity_noun_count >=1 and activity_exlusion_count == 0:
    	test.at[i, 'sedentary'] = 1
    	print item
    	#print stemmed_tokens
    	print 'verb: ',  re.findall(activity_verbs_regex, ' '.join(stemmed_tokens))
    	print 'noun: ', re.findall(activity_nouns_regex, ' '.join(stemmed_tokens))
        print 'exclude; ', re.findall(sedentary_exclusion_regex, ' '.join(stemmed_tokens))
    	print '\n'
    else:
    	test.at[i, 'sedentary'] = 0      
        
        
        
        
        
        

@TheFortressTurf @cranky_titz69 Muddin' runs in that family.
verb:  [u'run ']
noun:  [u'cran']
exclude;  []


Here’s hoping the #NHLJets break the Sportsnet curse. We always play bad on national tv.
#GoJetsGo
verb:  [u'play']
noun:  [u'hope', u'sport']
exclude;  []


@happyrunningmom Solid run yesterday Sherry. We are getting the same mild temps her too.  Enjoy it while it lasts.… https://t.co/vIRvTQjXYk
verb:  [u'run ']
noun:  [u'sol']
exclude;  []


On Safari in Western Cape South Africa is Plettenberg Bay Beaches Golf Hiking &amp; Fun 4 ALL Family Click https://t.co/B0620s4osi Qs Contact Us
verb:  [u'hike']
noun:  [u'golf']
exclude;  []


@SI_PeterKing Missing Lynch and any run blocking up front.
verb:  [u'run ']
noun:  [u'bloc']
exclude;  []


That was a roller coaster. The #Seahawks give up multiple big plays but still win. That's a big confidence boost. The Hawks are 5-2. #NFL
verb:  [u'play']
noun:  [u'roller']
exclude;  []


@PandasSoccer great ball handling lots of effort grea

verb:  [u'play']
noun:  [u'sport']
exclude;  []


It was nice watching the #WalkingDead and #TalkingDead this evening after wasting most of my day playing #Zombi on the #PS4. I did't beat it
verb:  [u'play']
noun:  [u'walking']
exclude;  []


That camera angle inside the Astros train that FOX just used was so damn cool
verb:  [u'train']
noun:  [u'fox']
exclude;  []


Corbetts Warehouse Sale is on from Nov 16 - 19 at the International Centre Save up to 80% off ski + snowboard gear… https://t.co/rR0tA2P1yC
verb:  [u'ski ']
noun:  [u'snowboard']
exclude;  []


Oh god damn I try but yknow all these other stupid people got no brain don’t think before they run their dirty ass… https://t.co/hztADX7oao
verb:  [u'run ']
noun:  [u'other']
exclude;  []


Astros score 4 in the 7th including solo HR by Springer and 2 run HR by Correa,lead 11-7
@cllrainslie @marlawd @artgypsychick @statu287
verb:  [u'run ']
noun:  [u'sol']
exclude;  []


Ferlands is turning into a beast of a hockey player!! Go Garla

verb:  [u'train']
noun:  [u'hope']
exclude;  []


@ckarras In Ontario, it is already illegal to smoke on and within 20 metres of children’s playgrounds and publicly owned sport fields
verb:  [u'play']
noun:  [u'sport']
exclude;  []


@CurlingJeff @bikeviewca Hmmm....🤔

We may have a problem then.
verb:  [u'bike']
noun:  [u'curling']
exclude;  []


OPDL Trials have begun. 
Any players born in 2003/04/05 looking to join our provincial program… https://t.co/ekb4rvM3IC
verb:  [u'play']
noun:  [u'trial']
exclude;  []


Ready to be a team player? See the latest opening: Entry Level Event Planning &amp; Event Coordination https://t.co/dtmSHTTupJ #entrylevel
verb:  [u'play']
noun:  [u'team play']
exclude;  []


I hope ppl keep doubting and disrespecting the #sens. It will make it a lot sweeter when we make it to the playoffs and go deep again
verb:  [u'play']
noun:  [u'hope']
exclude;  []


Just a run in the park on a lovely sunny fall day. 😁😁🏃💪🏽👟🌤#fall #fallcolors #beautifulday #smile… https:

EARLY DECEMBER AERIAL YOGA TEACHER TRAINING!
We moved the date back... so if you were interested… https://t.co/6rEkm72Pk3
verb:  [u'train']
noun:  [u'yoga']
exclude;  []


@NewsLiverpool Not again!LFC going after players that won't sign for the club.It's depressing to fans when LFC don'… https://t.co/mD7GsHt6nP
verb:  [u'play']
noun:  [u'club']
exclude;  []


.@CraigJButton on what likes about Gaudette: How about everything? He can skate, a really smart player. He makes plays, he can score goals
verb:  [u'play', u'play']
noun:  [u'skate']
exclude;  []


Very good start for a player who is far from being labeled a bust. Hope it continues. https://t.co/Y27snPGthg
verb:  [u'play']
noun:  [u'hope']
exclude;  []


Our @HEOMidgetAAA team plays the @KanataLasersU18 5:30pm at Richcraft @Sensplex tonight!!
verb:  [u'play']
noun:  [u'team play']
exclude;  []


I'm at McLean Park &amp; Playground in Richmond, British Columbia https://t.co/lIhG139rZu
verb:  [u'play']
noun:  [u'park']
exclude;  []


@razthematazz @travelling4life @blindjaywalker @jjjsummers @RoboRocket0 any team playing the leafs is the team i'm cheering for.
verb:  [u'play']
noun:  [u'team play']
exclude;  []


That is... not a good way to play hockey.
verb:  [u'play']
noun:  [u'hockey']
exclude;  []


@Sportsnet @EricEngels The Canadians need at least 4 NHL Dmen.They need 2Cs they need AG27 to play top mins They ne… https://t.co/pIuBHWUxJO
verb:  [u'play']
noun:  [u'sport']
exclude;  []


@tonygoldwyn what sport did you play while growing up? :) #scandal
verb:  [u'play']
noun:  [u'sport']
exclude;  []


You know William Nylander is tired when he skates at the same speed as all the other players. #TMLtalk
verb:  [u'play']
noun:  [u'skate', u'other']
exclude;  []


Is this the new dick sucking? Ball.

"HEEEE LOVEEEES TO REBOUND!!!!"

well he's a basketball player, he does that.
verb:  [u'play']
noun:  [u'ball']
exclude;  []


I have no idea who any of these players are.

I know Lillard and Nurkic.
And I know Lopez

...

I'm at McLean Park &amp; Playground in Richmond, British Columbia https://t.co/J3AohtCKGM
verb:  [u'play']
noun:  [u'park']
exclude;  []


First person I run in to @socscihist conf in Mtl is none other then #YorkU @FuyukiKurasawa
verb:  [u'run ']
noun:  [u'other']
exclude;  []


When, where, why and to who pass the ball is purely interaction between two players. A MUST on younger ages. What i… https://t.co/0nx8VTAv2x
verb:  [u'play']
noun:  [u'ball']
exclude;  []


Birds of Chicago playing in St. Albert tonight and they’re amazing...but I can’t go. Anyone need 3 tickets for tonight? Cheap! #yegmusic
verb:  [u'play']
noun:  [u'chicago']
exclude;  []


@NancyWilson I've known you were a great player since I saw Heart open for ZZ Top in Vancouver in 1975. #Respect #RoadcaseRoyale
verb:  [u'play']
noun:  [u'respect']
exclude;  []


We played some soccer today; all #WynnFitnessClubs against each other. #teambuilding #somuchfun See more:… https://t.co/3wKso2WTVG
verb:  [u'play']
noun:  [u'ot

@calstampeders Hope the owners will bring in winners next year to Calgary. This group has quit playing and so I am… https://t.co/4XPyIWrZWF
verb:  [u'play']
noun:  [u'hope']
exclude;  []


The Terry Fox Run is a popular event in Canada to raise funds to help cure canxer... Terry Fox was an athlete who l… https://t.co/IdgnPwCl5E
verb:  [u'run ']
noun:  [u'fox', u'fox']
exclude;  []


Despite being short on players today, the Lancers girls hockey team extends their winning streak with a 2-1 victory over Sturgeon Heights.
verb:  [u'play']
noun:  [u'hockey']
exclude;  []


@Isuckatpicking It’s the Oilers rule of hockey to let the returning player score.
verb:  [u'play']
noun:  [u'hockey']
exclude;  []


When a stranger approaches on the sidewalk, screaming  for no reason and running to the other side of the street. #GotMeIntoTroubl
verb:  [u'run ']
noun:  [u'other']
exclude;  []


@selenagomez you could do worse tgan Taylor Hall. Solid young man. Ask Carrie Underwood about hockey players.


verb:  [u'play', u'play']
noun:  [u'hope']
exclude;  []


Grade four recorder club playing ROCK! BOOM! BAG! @RedWillowPS https://t.co/ze5GxgLw84
verb:  [u'play']
noun:  [u'club']
exclude;  []


@Vertigo_Games How to enter: Post a photo of you trying PS VR, tag us at @PlayStationCA, and include hashtags #ITriedPSVR and #contest
verb:  [u'play']
noun:  [u'tag']
exclude;  []


@dailyblastlive Better hope those 3 UCLA players who shoplifted do pay the price for shoplifting. When u shoplift, u are busted
verb:  [u'play']
noun:  [u'hope']
exclude;  []


Happy #nationalphilanthropyday! You help us build new parks, playgrounds, mazes, ball diamonds, trails, gardens, ou… https://t.co/ARkwMmxnRP
verb:  [u'play']
noun:  [u'park', u'ball']
exclude;  []


I dont play that 0 respect now, real niggas move in silence
verb:  [u'play']
noun:  [u'respect']
exclude;  []


@Alan_Tovar12 @GeniusFootball Not talking about a player. The Quote is talking about 4 clubs (Chelsea/Dortmund/Man… https://t.co/LXBlOf

exclude;  []


Meet Bliss Valley, home to The Heart Of The Mountain ski shop &amp; Inn… https://t.co/ks3LWjSdLK
verb:  [u'ski ']
noun:  [u'mountain']
exclude;  []


I started listening to @MikeAndMike when I started running 4 yrs ago and one of the discussions I vividly remember… https://t.co/kxJG41gbtm
verb:  [u'run ']
noun:  [u'discus']
exclude;  []


Can’t thank @KitchFire enough for giving myself &amp;all the other women this opportunity to train together! Being out… https://t.co/6dANN8fjp8
verb:  [u'train']
noun:  [u'other']
exclude;  []


It is going to be interesting to see how this plays out with respect to certain people. https://t.co/M0G72Ql0Pv
verb:  [u'play']
noun:  [u'respect']
exclude;  []


“It kinda makes sense that there’s a Polynesian playing #Aquaman,” @PrideofGypsies explaining the smartest casting… https://t.co/cMJaFwHCvE
verb:  [u'play']
noun:  [u'casting']
exclude;  []


Toys loosing out to electronics! 🙁 I’m going to go play with my blocks now... 🏰 https://t.co/

verb:  [u'play']
noun:  [u'hockey']
exclude;  []


Getting ready to play Terror in Meeple City on @globalwinnipeg at the Bronx Park Community Centre. #winnipeg https://t.co/v9YhK0ykxr
verb:  [u'play']
noun:  [u'park']
exclude;  []


@jaimestein I've run into the same thing on instagram and am hoping they tweak the freshness of their algorithm soon
verb:  [u'run ']
noun:  [u'hope']
exclude;  []


